<a href="https://colab.research.google.com/github/suryaph971/Langchain/blob/main/01_A_LangChain_Deep_Dive_LLMs_Prompts_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Dive into LangChain


#LLMs, Prompt Templates, Caching, Streaming, Chains

In [1]:
pip install -r /content/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 657.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [2]:
pip install --upgrade -q langchain langchain-community

In [3]:
pip install --upgrade -q openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 kB 2.1 MB/s eta 0:00:00


In [4]:
pip show langchain

Name: langchain
Version: 0.3.26
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community


#Python-dotenv

In [26]:
import os
from dotenv import load_dotenv,find_dotenv

#loading the API key from .env
load_dotenv(find_dotenv(),override=True)
os.environ.get('OPENAI_API_KEY')

'sk-5678mnopqrstuvwx5678mnopqrstuvwx5678mnop'

#Chat Models: GPT-3.5 Turbo and GPT-4

In [27]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0.1)
prompt = "Explain about Data Engineering in one sentence"
output = llm.invoke(prompt)
print(output.content)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-5678m*******************************mnop. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [29]:
#use chat completion api messages: system,Assistant and Human
from langchain_openai import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    AIMessage,
    HumanMessage
)
messages=[
    SystemMessage(content="You are physical assisant, so please respond in English and Telugu"),
    HumanMessage(content="Explain about Data Engineering in one sentence")
]

output = llm.invoke(messages)
print(output.content)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-5678m*******************************mnop. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

#Caching LLM Responses

In [30]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain.cache import InMemoryCache
llm = ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0.1)
set_llm_cache(InMemoryCache())

In [32]:
%%time
llm.invoke("Explain about Data Engineering in one sentence")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-5678m*******************************mnop. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [34]:
#To see the performance of caching
%%time
llm.invoke("Explain about Data Engineering in one sentence")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-5678m*******************************mnop. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

#SQLite Caching

In [35]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [36]:
%%time
llm.invoke("Explain about Data Engineering in one sentence")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-5678m*******************************mnop. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [37]:
#To see the performance of caching
%%time
llm.invoke("Explain about Data Engineering in one sentence")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-5678m*******************************mnop. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

#LLM Streaming

In [38]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-5678m*******************************mnop. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [39]:
for chunk in llm.stream(prompt):
    print(chunk.content,end="",flush=True)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-5678m*******************************mnop. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

#PromptTemplates

In [42]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define a template for the prompt
template = '''You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate(
    input_variables=["virus","language"],
    template=template
)

prompt = prompt_template.format(virus="HIV",language="English")
prompt

'You are an experienced virologist.\nWrite a few sentences about the following virus "HIV" in English.'

In [43]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-5678m*******************************mnop. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

#ChatPromptTemplates

In [47]:
from langchain.prompts import ChatPromptTemplate,HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You respond only in the JSON format"),
        HumanMessagePromptTemplate.from_template('Top {n} counts in {area} by Population'
        )
    ]
)

messages = chat_template.format_messages(n='5',area="World")
messages


[SystemMessage(content='You respond only in the JSON format', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Top 5 counts in World by Population', additional_kwargs={}, response_metadata={})]

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)